In [3]:
from dotenv import load_dotenv
import os
from pinecone import Pinecone, ServerlessSpec
from sentence_transformers import SentenceTransformer
import openrouter
from openai import OpenAI

load_dotenv()


True

In [4]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=384, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '1d70e2e3d239bdbbe1726783b5fa40e1', 'Date': 'Fri, 06 Sep 2024 06:12:20 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [5]:
import json
data = json.load(open("reviews.json"))
data

{'reviews': [{'professor': 'Dr. Emily Chen',
   'subject': 'Mathematics',
   'stars': 4,
   'review': 'Professor Chen is an excellent math instructor who makes complex concepts easy to understand. Her lectures are well-organized and she provides plenty of practice problems to help students master the material. She is also very responsive to student questions and concerns.'},
  {'professor': 'Prof. Liam Nguyen',
   'subject': 'Political Science',
   'stars': 3,
   'review': 'Professor Nguyen is knowledgeable about political science, but his lectures can sometimes be dry and difficult to stay engaged with. However, he is fair in his grading and provides constructive feedback on assignments. He is also willing to meet with students during office hours to provide additional support.'},
  {'professor': 'Dr. Sophia Patel',
   'subject': 'Chemistry',
   'stars': 5,
   'review': "Dr. Patel is an exceptional chemistry professor who is passionate about the subject and dedicated to her students' 

In [6]:
hf_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

openrouter_client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=os.getenv("OPENROUTER_API_KEY")
)

/Applications/anaconda3/envs/rag/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [13]:
process_data = []

for review in data['reviews']:
    # Generate embeddings using Hugging Face
    embedding = hf_model.encode(review['review'], convert_to_numpy=True)  # Generate the embedding
    
    # Append the embedding and metadata to the process_data list
    process_data.append({
        "values": embedding.tolist(),  # Convert to list since numpy arrays may not serialize easily
        "id": review['professor'],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })


In [44]:
process_data[1]

{'values': [-0.04652659595012665,
  0.016679229214787483,
  -0.040222786366939545,
  -0.03891471400856972,
  -0.021088983863592148,
  0.031280867755413055,
  0.026171401143074036,
  -0.009779615327715874,
  -0.019406430423259735,
  0.0019631129689514637,
  0.012521269731223583,
  -0.01873900555074215,
  -0.09775739163160324,
  0.0426701083779335,
  0.011120659299194813,
  0.024220885708928108,
  -0.02887388877570629,
  -0.011839131824672222,
  -0.04837306588888168,
  -0.06383007764816284,
  -0.014969553798437119,
  0.010058365762233734,
  0.03395537659525871,
  -0.008895643055438995,
  0.0330820307135582,
  -0.08633878827095032,
  0.01012332085520029,
  -0.027989570051431656,
  0.0837399810552597,
  -0.009977707639336586,
  -0.000494216161314398,
  0.03168300539255142,
  -0.021679511293768883,
  0.010645185597240925,
  0.04821894317865372,
  0.012601551599800587,
  -0.003584157209843397,
  0.05304519832134247,
  0.04119936376810074,
  0.03906657174229622,
  0.0036177148576825857,
  0.0

In [14]:
index = pc.Index('rag')
index.upsert(
    vectors=process_data,
    namespace="ns1"
)

{'upserted_count': 20}

In [15]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}